In [100]:
import pandas as pd
import json
import numpy as np

In [101]:
from key_features import *

In [102]:
fm = pd.read_csv('data/fm_04092022.csv')

In [103]:
fm

,UID,Name,Nat,Based,Club,Position,Preferred Foot,Right Foot,Left Foot,Height,...,Bal_20,Kic_20,Sta_20,Agi_20,Wor_20,Bra_20,Cmd_20,Fin_20,1v1_20,TRO_20
0,7458500,Lionel Messi,ARG,France (Ligue 1),PSG,"AM (RC), ST (C)",Left,Fairly Strong,Very Strong,169 cm,...,19,1,13,19,7,10,3,20,3,1
1,18004457,Kevin De Bruyne,BEL,England (Premier League),Manchester City,"M (RLC), AM (C)",Either,Very Strong,Strong,181 cm,...,14,4,17,13,15,14,2,16,2,1
2,735216,Cristiano Ronaldo,POR,England (Premier League),Man Utd,"AM (L), ST (C)",Either,Very Strong,Strong,187 cm,...,14,2,15,13,10,16,4,19,2,2
3,85139014,Kylian Mbappé,FRA,France (Ligue 1),PSG,"AM (RL), ST (C)",Right,Very Strong,Reasonable,178 cm,...,14,3,14,16,10,12,1,17,3,3
4,98028755,Mohamed Salah,EGY,England (Premier League),Liverpool,"AM (RL), ST (C)",Left,Weak,Very Strong,175 cm,...,17,1,14,16,14,11,3,18,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105670,7801043,Jonathan Xu,SIN,Singapore (NFL League One),Project Vaults,D (C),Right Only,Very Strong,Weak,180 cm,...,6,1,5,5,6,12,3,4,3,2
105671,67103072,Christopher,VEN,Venezuela,NaN,ST (C),Right Only,Very Strong,Weak,189 cm,...,3,1,8,11,3,1,1,11,3,3
105672,23277225,Sultan Ashardeen,SIN,Singapore (NFL League Two),Balestier Utd,M/AM (LC),Left Only,Weak,Very Strong,160 cm,...,7,3,8,11,12,7,4,7,2,2
105673,23487831,Satria Salimi,SIN,Singapore (NFL League Two),Jungfrau Punggol,D/WB (R),Left Only,Weak,Very Strong,178 cm,...,3,1,4,11,8,3,2,2,2,2


In [104]:
fm.iloc[136]

UID               315542
Name        Daniel Alves
Nat                  BRA
Based     Spain (LaLiga)
Club           Barcelona
               ...      
Bra_20                17
Cmd_20                 3
Fin_20                12
1v1_20                 2
TRO_20                 1
Name: 136, Length: 161, dtype: object

In [105]:
def transfer_value_to_float(value):
    if 'M' in value:
        if '.' in value:
            return value.replace('M', '').replace('.','') + '00000'
        else:
            return value.replace('M', '') + '000000'
    elif 'K' in value:
        if '.' in value:
            return value.replace('K', '').replace('.','') + '00'
        else:
            return value.replace('K', '') + '000'
    else:
        return value


In [106]:
def transfer_value_prep():
    fm['Transfer Value'] = fm['Transfer Value'].apply(lambda x: x.split(' ')[0])
    fm['Transfer Value'] = fm['Transfer Value'].apply(lambda x: x.replace('€', ''))
    fm['Transfer Value'] = fm['Transfer Value'].apply(lambda x: transfer_value_to_float(x))
    fm['Transfer Value'] = fm['Transfer Value'].apply(lambda x: np.nan if x == 'Not' else x)
    fm['Transfer Value'] = fm['Transfer Value'].astype(float)

In [107]:
def wage_prep():
    fm['Wage'] = fm['Wage'].fillna('-')
    fm['Wage'] = fm['Wage'].apply(lambda x: x.split(' ')[0])
    fm['Wage'] = fm['Wage'].apply(lambda x: x.replace('€', '').replace(',',''))
    fm['Wage'] = fm['Wage'].apply(lambda x: np.nan if x == '-' else x)
    fm['Wage'] = fm['Wage'].astype(float) * 12

In [108]:
wage_prep()

In [109]:
transfer_value_prep()

In [110]:
fm['Wage']

0         73800000.0
1         20676000.0
2         29196000.0
3         25104000.0
4         13680000.0
             ...    
105670       20400.0
105671           NaN
105672        5700.0
105673        3600.0
105674        4200.0
Name: Wage, Length: 105675, dtype: float64

In [111]:
fm['Transfer Value']

0                 NaN
1         248000000.0
2                 NaN
3         249000000.0
4         136000000.0
             ...     
105670            0.0
105671            0.0
105672            0.0
105673            0.0
105674            0.0
Name: Transfer Value, Length: 105675, dtype: float64

In [112]:
fm.Caps.fillna(0, inplace=True)
fm['Caps'] = fm['Caps'].apply(lambda x: True if x != 0 else False)

In [113]:
fm.to_csv('data/fm_13092022.csv', index=False)

***

In [5]:
{'key': GK['Sweeper Keeper (Defend)'][0], 'preferred': GK['Sweeper Keeper (Defend)'][1]}

{'key': [' Ref ',
  ' Pos ',
  ' Ant ',
  ' Cnt ',
  ' Kic ',
  ' Agi ',
  ' Cmd ',
  ' 1v1 '],
 'preferred': [' Vis ',
  ' Cmp ',
  ' Pas ',
  ' Dec ',
  ' Fir ',
  ' Com ',
  ' Acc ',
  ' Aer ',
  ' Thr ',
  ' Han ',
  ' TRO ']}

In [6]:
for key, value in GK.items():
    GK[key] = {'key': value[0], 'preferred':value[1]}

In [7]:
for key, value in DEF.items():
    DEF[key] = {'key': value[0], 'preferred':value[1]}

In [8]:
for key, value in MID.items():
    MID[key] = {'key': value[0], 'preferred':value[1]}

In [9]:
for key, value in FW.items():
    FW[key] = {'key': value[0], 'preferred':value[1]}

In [10]:
positions = {'GK': GK, 'DEF': DEF, 'MID': MID, 'FW': FW}

In [11]:
for key, value in positions.items():
    for role, attributes in value.items():
        attributes['key'] = [x.strip() for x in attributes['key']]
        attributes['preferred'] = [x.strip() for x in attributes['preferred']]

In [12]:
json.dump(positions,indent=2,fp=open('data/positions.json','w'))

In [13]:
scores = pd.DataFrame()
scores['UID'] = fm['UID']
scores['Name'] = fm.Name
scores.head()

,UID,Name
0,7458500,Lionel Messi
1,18004457,Kevin De Bruyne
2,735216,Cristiano Ronaldo
3,85139014,Kylian Mbappé
4,98028755,Mohamed Salah


In [14]:
all_attributes = []

for key, value in positions.items():
    for role, attributes in value.items():
        all_attributes.extend(attributes['key'])
        all_attributes.extend(attributes['preferred'])

In [15]:
all_attributes = list(set(all_attributes))
len(all_attributes)

39

In [16]:
NON_GK_ATRIBUTES = [x.strip() for x in NON_GK_ATRIBUTES]
NON_PLAYER_ATTRIBUTES = [x.strip() for x in NON_PLAYER_ATTRIBUTES]
NON_GK_ATRIBUTES

['L Th', 'Lon', 'Tck', 'Cro', 'Mar', 'Cor', 'Hea', 'Dri', 'Fin']

In [17]:
for pos_name, roles in positions.items():
    for role_name, attributes in roles.items():
        keys_22 = attributes['key']
        preferred_22 = attributes['preferred']
        keys_21 = [x + '_21' for x in keys_22]
        keys_20 = [x + '_20' for x in keys_22]
        preferred_21 = [x + '_21' for x in preferred_22]
        preferred_20 = [x + '_20' for x in preferred_22]

        if pos_name == 'GK':
            others_22 = [x for x in all_attributes if x not in keys_22 and x not in preferred_22 and x not in NON_GK_ATRIBUTES]
        else:
            others_22 = [x for x in all_attributes if x not in keys_22 and x not in preferred_22 and x not in NON_PLAYER_ATTRIBUTES]

        others_21 = [x + '_21' for x in others_22]
        others_20 = [x + '_20' for x in others_22]
        scores[role_name] = (fm[keys_22].mean(axis=1) * 0.55 + fm[preferred_22].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.6\
                            +(fm[keys_21].mean(axis=1) * 0.55 + fm[preferred_21].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.3\
                            +(fm[keys_20].mean(axis=1) * 0.55 + fm[preferred_20].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.1

        print(f"{pos_name} {role_name} done")

GK Sweeper Keeper (Defend) done
GK Sweeper Keeper (Support) done
GK Sweeper Keeper (Attack) done
GK Goalkeeper (Defend) done
DEF Ball Playing Defender (Defender) done
DEF Ball Playing Defender (Stopper) done
DEF Ball Playing Defender (Cover) done
DEF No-Nonsense Centre-Back (Defend) done
DEF No-Nonsense Centre-Back (Stopper) done
DEF No-Nonsense Centre-Back (Cover) done
DEF Wide Center-Back (Defend) done
DEF Wide Center-Back (Support) done
DEF Wide Center-Back (Attack) done
DEF Libero (Support) done
DEF Libero (Attack) done
DEF Central Defender (Defend) done
DEF Central Defender (Stopper) done
DEF Central Defender (Cover) done
DEF Full-Back (Defend) done
DEF Full-Back (Support) done
DEF Full-Back (Attack) done
DEF Full-Back (Automatic) done
DEF Wing-Back (Defend) done
DEF Wing-Back (Support) done
DEF Wing-Back (Attack) done
DEF Wing-Back (Automatic) done
DEF No-Nonsense Full-Back (Defend) done
DEF Complete Wing-Back (Support) done
DEF Complete Wing-Back (Attack) done
DEF Inverted Wing-

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
scaler = MinMaxScaler((0, 100))
scores.iloc[:,2:] = pd.DataFrame(scaler.fit_transform(scores.iloc[:,2:]), columns=scores.iloc[:,2:].columns)

In [20]:
scores

,UID,Name,Sweeper Keeper (Defend),Sweeper Keeper (Support),Sweeper Keeper (Attack),Goalkeeper (Defend),Ball Playing Defender (Defender),Ball Playing Defender (Stopper),Ball Playing Defender (Cover),No-Nonsense Centre-Back (Defend),...,Target Forward (Attack),Pressing Forward (Defend),Pressing Forward (Support),Pressing Forward (Attack),Deep Lying Forward (Support),Deep Lying Forward (Attack),Poacher (Attack),Advanced Forward (Attack),Trequartista (Attack),False Nine (Support)
0,7458500,Lionel Messi,48.598259,50.900626,48.621327,37.991271,71.782208,77.580943,79.672841,50.398150,...,79.490255,82.398546,84.396432,82.329516,100.000000,100.000000,99.253574,100.000000,100.000000,100.000000
1,18004457,Kevin De Bruyne,47.071123,48.849252,46.709940,37.049992,75.875811,80.618178,80.605860,60.925762,...,75.966042,85.033273,87.209242,84.736319,90.668108,90.018860,83.675226,89.982949,84.152873,86.275898
2,735216,Cristiano Ronaldo,41.105145,41.856870,40.209627,34.496707,72.589088,75.269962,76.346184,65.096741,...,83.322316,74.145618,75.809261,76.749777,84.375619,82.920869,96.422424,85.262985,81.275403,79.924705
3,85139014,Kylian Mbappé,44.851561,46.977652,45.093840,34.934224,62.458683,66.191743,70.365017,48.755467,...,73.136851,82.140336,83.340182,83.735965,86.776450,87.006150,95.284344,96.192943,88.596203,87.766337
4,98028755,Mohamed Salah,47.501334,48.460738,47.017198,36.679734,73.873958,76.662360,78.879207,63.715736,...,80.293873,88.921138,90.228671,90.152609,90.856627,90.289463,92.198532,95.531873,89.224046,89.790820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105670,7801043,Jonathan Xu,16.262681,15.675351,14.893391,17.454431,35.569064,36.407481,33.533979,41.551618,...,32.328889,30.062232,30.859875,30.466701,27.738268,27.471915,31.205873,27.843652,24.046561,24.717146
105671,67103072,Christopher,7.096397,7.677285,7.458001,6.690852,19.076253,21.371481,17.449288,18.545291,...,27.905681,19.326234,19.296103,22.963766,30.059897,30.146781,40.503706,34.327125,29.075070,28.766112
105672,23277225,Sultan Ashardeen,5.883961,5.979715,5.393650,6.099699,6.214953,7.501561,7.117930,6.254800,...,6.054053,15.411895,14.665386,15.154153,14.454079,15.052071,14.468369,21.404774,16.687134,17.085442
105673,23487831,Satria Salimi,7.624824,7.307348,6.902010,7.805731,12.889805,14.105919,12.026349,16.923892,...,8.420526,16.412705,15.388845,14.605953,11.796251,11.690037,12.360814,15.230194,12.208635,12.826049


In [21]:
searched_ids = fm[(fm['Based'] == 'Turkey (Süper Lig)')]['UID'].values

In [22]:
selected_role = 'Defensive Midfielder (Defend)'
scores.loc[scores['UID'].isin(searched_ids),['Name',selected_role]].sort_values(by=selected_role, ascending=False).head(10)

,Name,Defensive Midfielder (Defend)
34341,Atiba Hutchinson,89.837296
34324,Souza,88.964646
34367,Luiz Gustavo,85.222284
35298,Benjamin Stambouli,83.643629
35164,Lucas Biglia,83.620334
34531,Hakan Arslan,83.071120
34298,Miralem Pjanić,82.782626
35312,Mahmut Tekdemir,82.472629
34317,Marcão,81.912663
35042,André Poko,81.878617


In [23]:
#scores.to_csv('data/scores_13092022.csv', index=False)

***

In [24]:
import numpy as np

In [25]:
managers = pd.read_csv('data/fm22managers.csv')
managers.columns = [x.strip() for x in managers.columns]
managers = managers.applymap(lambda x: x.strip() if isinstance(x, str) else x)
managers = managers.applymap(lambda x: np.nan if x=='' else x)
managers = managers.dropna(subset=['Club'])
managers = managers.drop_duplicates(subset=['Club'], keep='first')

In [26]:
miguel_crespo = fm[fm['Name'] == 'Miguel Crespo']

In [27]:
miguel_crespo

,UID,Name,Nat,Based,Club,Position,Preferred Foot,Right Foot,Left Foot,Height,...,Bal_20,Kic_20,Sta_20,Agi_20,Wor_20,Bra_20,Cmd_20,Fin_20,1v1_20,TRO_20
34748,84122771,Miguel Crespo,POR,Turkey (Süper Lig),Fenerbahçe,"DM, M (C)",Right,Very Strong,Reasonable,185 cm,...,13,1,13,12,12,10,2,8,2,1


In [28]:
scores[scores['UID'] == miguel_crespo.UID.values[0]]

,UID,Name,Sweeper Keeper (Defend),Sweeper Keeper (Support),Sweeper Keeper (Attack),Goalkeeper (Defend),Ball Playing Defender (Defender),Ball Playing Defender (Stopper),Ball Playing Defender (Cover),No-Nonsense Centre-Back (Defend),...,Target Forward (Attack),Pressing Forward (Defend),Pressing Forward (Support),Pressing Forward (Attack),Deep Lying Forward (Support),Deep Lying Forward (Attack),Poacher (Attack),Advanced Forward (Attack),Trequartista (Attack),False Nine (Support)
34748,84122771,Miguel Crespo,37.806093,38.206481,37.063886,29.734976,65.896695,67.871172,67.173959,59.976712,...,64.568067,70.895761,71.514808,69.992754,70.376955,70.628946,64.622923,70.928646,64.574433,67.023361


In [29]:
managers[managers['Club'] == miguel_crespo.Club.values[0]]

,UID,Name,Nat,Age,Club,Youth,Marking Style,Preferred Attacking Formation,Preferred Defensive Formation,Second Preferred Formation,Pressing Style,Playing Style,Tactical Style,Playing Mentality,Preferred Formation
4791,8480793,İsmail Kartal,TUR,60,Fenerbahçe,10,Mixed,4-4-2,4-4-1-1,4-4-1-1,Balanced,Standard,Park the Bus,Cautious,4-4-1-1
